In [30]:
import os

import numpy as np

from redisvl.query import VectorQuery
from redisvl.query.filter import Tag
from redisvl.index import SearchIndex

from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

os.environ["REDIS_URL"] = "redis://localhost:6379"

In [35]:
def get_vec(prompt, length=512):
    inputs = tokenizer(
                prompt, padding=True, truncation=True, return_tensors="pt", max_length=length
            )
    with torch.no_grad():
        embed = model(**inputs).last_hidden_state[:, 0].cpu().detach().numpy()
    return embed

In [32]:
prompt = "Summarize the opinion of PualG on hacker news"
embedding = get_vec(prompt)

In [33]:
# get the index
conn_url = "redis://localhost:6379"
story_index = SearchIndex.from_existing("story_index", redis_url=conn_url)

In [43]:
v = VectorQuery(embedding,
                "doc_embedding",
                return_fields=[])


results = story_index.query(v)
result_print(results)

ResponseError: Error parsing vector similarity query: query vector blob size (1536) does not match index's expected size (2048).